# 競馬予想アプリ - 推論専用ノートブック

このノートブックは**推論（予測）のみ**を行います。

## ハイブリッド構成

```
┌─────────────────┐     ┌──────────────────┐
│  ローカルFastAPI │     │   Supabase       │
│  - モデル学習    │────▶│  - モデル保存    │
│  - 特徴量生成    │     │  - バージョン管理 │
└─────────────────┘     └────────┬─────────┘
                                 │
                        ┌────────▼─────────┐
                        │   Google Colab   │
                        │  - モデル読込    │ ← このノートブック
                        │  - 当日予測実行  │
                        └──────────────────┘
```

## 使用方法
1. 左サイドバーの🔑からシークレットに以下を設定：
   - `SUPABASE_URL`
   - `SUPABASE_KEY`
2. セルを順番に実行

## 前提条件
- ローカルFastAPIで学習済みモデルがSupabase Storageにアップロードされていること
- Supabase Storageに `models` バケットが作成されていること

## 1. 環境セットアップ

In [ ]:
# 必要なパッケージをインストール
!pip install -q supabase requests beautifulsoup4 lxml pandas numpy lightgbm scikit-learn

In [ ]:
# Supabase認証情報の設定
try:
    from google.colab import userdata
    SUPABASE_URL = userdata.get('SUPABASE_URL')
    SUPABASE_KEY = userdata.get('SUPABASE_KEY')
    print("Colabシークレットから認証情報を取得しました")
except:
    # 直接入力（テスト用）
    SUPABASE_URL = "https://your-project.supabase.co"  # @param {type:"string"}
    SUPABASE_KEY = "your-service-role-key"  # @param {type:"string"}
    print("手動入力の認証情報を使用します")

# 接続テスト
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"Supabase接続: {SUPABASE_URL[:30]}...")

## 2. モデルのダウンロード

In [ ]:
import pickle
import io
import pandas as pd
import numpy as np

BUCKET_NAME = "models"

def list_available_models():
    """利用可能なモデル一覧を取得"""
    try:
        files = supabase.storage.from_(BUCKET_NAME).list()
        models = [f["name"] for f in files if f["name"].endswith(".pkl")]
        print(f"利用可能なモデル: {models}")
        return models
    except Exception as e:
        print(f"エラー: {e}")
        return []

def download_model(version: str = "v1"):
    """モデルをダウンロード"""
    filename = f"model_{version}.pkl"
    print(f"モデルをダウンロード中: {filename}")
    
    try:
        response = supabase.storage.from_(BUCKET_NAME).download(filename)
        buffer = io.BytesIO(response)
        model_data = pickle.load(buffer)
        print(f"✓ モデルをダウンロードしました")
        print(f"  バージョン: {model_data.get('model_version', 'unknown')}")
        print(f"  特徴量数: {len(model_data.get('feature_columns', []))}")
        return model_data
    except Exception as e:
        print(f"✗ ダウンロード失敗: {e}")
        return None

# 利用可能なモデル一覧を表示
list_available_models()

In [ ]:
# モデルをダウンロード
MODEL_VERSION = "v1"  # @param {type:"string"}

model_data = download_model(MODEL_VERSION)

if model_data:
    model = model_data["model"]
    scaler = model_data["scaler"]
    calibrator = model_data.get("calibrator")
    feature_columns = model_data["feature_columns"]
    print(f"\nモデル準備完了")
else:
    print("モデルのダウンロードに失敗しました")
    print("ローカルFastAPIでモデルを学習・アップロードしてください")

## 3. 当日レースデータの取得

In [ ]:
import requests
import time
import re
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
from typing import Optional, List, Dict

# スクレイピング設定
SCRAPE_INTERVAL = 1.5
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

JRA_COURSE_CODES = {"01", "02", "03", "04", "05", "06", "07", "08", "09", "10"}
COURSE_NAMES = {
    "01": "札幌", "02": "函館", "03": "福島", "04": "新潟",
    "05": "東京", "06": "中山", "07": "中京", "08": "京都",
    "09": "阪神", "10": "小倉",
}

session = requests.Session()
session.headers.update(HEADERS)
last_request_time = 0

def fetch_html(url: str) -> str:
    global last_request_time
    elapsed = time.time() - last_request_time
    if elapsed < SCRAPE_INTERVAL:
        time.sleep(SCRAPE_INTERVAL - elapsed)
    
    response = session.get(url, timeout=30)
    last_request_time = time.time()
    
    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"
    else:
        response.encoding = response.apparent_encoding or "utf-8"
    
    return response.text

def is_jra_race(race_id: str) -> bool:
    if len(race_id) >= 6:
        return race_id[4:6] in JRA_COURSE_CODES
    return False

print("スクレイピング関数を定義しました")

In [ ]:
def scrape_race_list(target_date: date, jra_only: bool = True) -> List[Dict]:
    """指定日のレース一覧を取得"""
    date_str = target_date.strftime("%Y%m%d")
    url = f"https://race.netkeiba.com/top/race_list.html?kaisai_date={date_str}"
    
    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")
    
    races = []
    seen_ids = set()
    
    for link in soup.find_all("a", href=True):
        href = link.get("href", "")
        match = re.search(r"race_id=(\d{12})", href)
        if match:
            race_id = match.group(1)
            if race_id not in seen_ids:
                if jra_only and not is_jra_race(race_id):
                    continue
                seen_ids.add(race_id)
                races.append({
                    "race_id": race_id,
                    "date": target_date.isoformat(),
                    "race_name": link.get_text(strip=True),
                })
    
    return races

def scrape_shutuba(race_id: str) -> Dict:
    """出馬表を取得（レース前のデータ）"""
    url = f"https://race.netkeiba.com/race/shutuba.html?race_id={race_id}"
    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")
    
    race_data = {
        "race_id": race_id,
        "course": COURSE_NAMES.get(race_id[4:6], ""),
        "race_number": int(race_id[10:12]) if len(race_id) >= 12 else 0,
    }
    
    # レース名
    title_elem = soup.select_one(".RaceName")
    if title_elem:
        race_data["race_name"] = title_elem.get_text(strip=True)
    
    # 距離・馬場
    race_data_elem = soup.select_one(".RaceData01")
    if race_data_elem:
        text = race_data_elem.get_text()
        
        distance_match = re.search(r"(\d+)m", text)
        if distance_match:
            race_data["distance"] = int(distance_match.group(1))
        
        if "芝" in text:
            race_data["track_type"] = "芝"
        elif "ダート" in text or "ダ" in text:
            race_data["track_type"] = "ダート"
    
    # 出走馬
    entries = []
    table = soup.select_one("table.Shutuba_Table")
    if table:
        for row in table.select("tr.HorseList"):
            entry = parse_shutuba_row(row)
            if entry:
                entries.append(entry)
    
    race_data["entries"] = entries
    return race_data

def parse_shutuba_row(row) -> Optional[Dict]:
    """出馬表の行をパース"""
    entry = {}
    
    # 枠番
    waku = row.select_one("td.Waku")
    if waku:
        try:
            entry["frame_number"] = int(waku.get_text(strip=True))
        except:
            pass
    
    # 馬番
    umaban = row.select_one("td.Umaban")
    if umaban:
        try:
            entry["horse_number"] = int(umaban.get_text(strip=True))
        except:
            return None
    
    # 馬名・ID
    horse_link = row.select_one("span.HorseName a")
    if horse_link:
        href = horse_link.get("href", "")
        match = re.search(r"/horse/(\d+)", href)
        if match:
            entry["horse_id"] = match.group(1)
        entry["horse_name"] = horse_link.get_text(strip=True)
    
    # 騎手
    jockey_link = row.select_one("td.Jockey a")
    if jockey_link:
        href = jockey_link.get("href", "")
        match = re.search(r"/jockey/(?:result/recent/)?(\d+)", href)
        if match:
            entry["jockey_id"] = match.group(1)
        entry["jockey_name"] = jockey_link.get_text(strip=True)
    
    # 斤量
    weight_elem = row.select_one("td.Txt_C")
    if weight_elem:
        try:
            entry["weight"] = float(weight_elem.get_text(strip=True))
        except:
            pass
    
    # オッズ
    odds_elem = row.select_one("td.Odds span")
    if odds_elem:
        try:
            odds_text = odds_elem.get_text(strip=True)
            if odds_text and odds_text != "---":
                entry["odds"] = float(odds_text)
        except:
            pass
    
    return entry if entry.get("horse_number") else None

print("出馬表スクレイピング関数を定義しました")

In [ ]:
# 対象日を設定
target_date = date.today()  # 今日
# target_date = date(2024, 12, 28)  # 特定の日付

print(f"対象日: {target_date}")

# レース一覧を取得
races = scrape_race_list(target_date, jra_only=True)
print(f"\n{len(races)}件のJRAレースが見つかりました")

for race in races[:5]:
    print(f"  - {race['race_id']}: {race['race_name']}")

## 4. 特徴量の生成

In [ ]:
def get_horse_stats(horse_id: str) -> Dict:
    """馬の過去成績を取得（Supabaseから）"""
    try:
        result = supabase.table("entries").select("result, odds").eq("horse_id", horse_id).order("id", desc=True).limit(10).execute()
        
        if not result.data:
            return {"win_rate": 0, "place_rate": 0, "avg_odds": 10}
        
        results = [r["result"] for r in result.data if r["result"]]
        odds_list = [r["odds"] for r in result.data if r["odds"]]
        
        if not results:
            return {"win_rate": 0, "place_rate": 0, "avg_odds": 10}
        
        win_rate = sum(1 for r in results if r == 1) / len(results)
        place_rate = sum(1 for r in results if r <= 3) / len(results)
        avg_odds = sum(odds_list) / len(odds_list) if odds_list else 10
        
        return {
            "win_rate": win_rate,
            "place_rate": place_rate,
            "avg_odds": avg_odds,
        }
    except:
        return {"win_rate": 0, "place_rate": 0, "avg_odds": 10}

def get_jockey_stats(jockey_id: str) -> Dict:
    """騎手の過去成績を取得（Supabaseから）"""
    try:
        result = supabase.table("entries").select("result").eq("jockey_id", jockey_id).order("id", desc=True).limit(50).execute()
        
        if not result.data:
            return {"jockey_win_rate": 0, "jockey_place_rate": 0}
        
        results = [r["result"] for r in result.data if r["result"]]
        
        if not results:
            return {"jockey_win_rate": 0, "jockey_place_rate": 0}
        
        win_rate = sum(1 for r in results if r == 1) / len(results)
        place_rate = sum(1 for r in results if r <= 3) / len(results)
        
        return {
            "jockey_win_rate": win_rate,
            "jockey_place_rate": place_rate,
        }
    except:
        return {"jockey_win_rate": 0, "jockey_place_rate": 0}

print("統計取得関数を定義しました")

In [ ]:
def create_features(race_data: Dict) -> pd.DataFrame:
    """レースデータから特徴量を生成"""
    entries = race_data.get("entries", [])
    if not entries:
        return pd.DataFrame()
    
    features_list = []
    
    for entry in entries:
        features = {
            "horse_number": entry.get("horse_number"),
            "frame_number": entry.get("frame_number", 0),
            "weight": entry.get("weight", 55),
            "odds": entry.get("odds", 10),
            "distance": race_data.get("distance", 1600),
            "track_type": 1 if race_data.get("track_type") == "芝" else 0,
        }
        
        # 馬の過去成績
        if entry.get("horse_id"):
            horse_stats = get_horse_stats(entry["horse_id"])
            features.update(horse_stats)
        else:
            features.update({"win_rate": 0, "place_rate": 0, "avg_odds": 10})
        
        # 騎手の過去成績
        if entry.get("jockey_id"):
            jockey_stats = get_jockey_stats(entry["jockey_id"])
            features.update(jockey_stats)
        else:
            features.update({"jockey_win_rate": 0, "jockey_place_rate": 0})
        
        features_list.append(features)
    
    df = pd.DataFrame(features_list)
    
    # 足りない特徴量を0で埋める
    for col in feature_columns:
        if col not in df.columns:
            df[col] = 0
    
    return df

print("特徴量生成関数を定義しました")

## 5. 予測の実行

In [ ]:
def predict_race(race_data: Dict) -> Dict:
    """レースの予測を実行"""
    df = create_features(race_data)
    
    if df.empty:
        return None
    
    # 特徴量を選択
    X = df[feature_columns].copy()
    X = X.fillna(0)
    
    # スケーリング
    X_scaled = pd.DataFrame(
        scaler.transform(X),
        columns=X.columns,
        index=X.index,
    )
    
    # 予測
    scores = model.predict(X_scaled)
    
    # 確率計算（ソフトマックス）
    exp_scores = np.exp(scores - np.max(scores))
    probabilities = exp_scores / exp_scores.sum()
    
    # キャリブレーション適用
    if calibrator is not None:
        pred_min = scores.min()
        pred_max = scores.max()
        if pred_max > pred_min:
            scores_normalized = (scores - pred_min) / (pred_max - pred_min)
            calibrated_probs = calibrator.predict(scores_normalized)
            total = calibrated_probs.sum()
            if total > 0:
                probabilities = calibrated_probs / total
    
    # 結果を整形
    results = []
    entries = race_data.get("entries", [])
    
    for i, entry in enumerate(entries):
        results.append({
            "horse_number": entry.get("horse_number"),
            "horse_name": entry.get("horse_name", "不明"),
            "score": float(scores[i]),
            "probability": float(probabilities[i]),
            "odds": entry.get("odds"),
            "expected_value": float(probabilities[i] * (entry.get("odds", 1) or 1)),
        })
    
    # スコア順にソート
    results.sort(key=lambda x: x["score"], reverse=True)
    
    # 順位付け
    for i, r in enumerate(results):
        r["pred_rank"] = i + 1
    
    return {
        "race_id": race_data["race_id"],
        "race_name": race_data.get("race_name", ""),
        "course": race_data.get("course", ""),
        "predictions": results,
    }

print("予測関数を定義しました")

In [ ]:
from tqdm.notebook import tqdm

# 全レースの予測を実行
all_predictions = []

for race_info in tqdm(races, desc="予測実行中"):
    try:
        # 出馬表を取得
        race_data = scrape_shutuba(race_info["race_id"])
        
        # 予測実行
        prediction = predict_race(race_data)
        
        if prediction:
            all_predictions.append(prediction)
            
    except Exception as e:
        print(f"エラー: {race_info['race_id']} - {e}")

print(f"\n{len(all_predictions)}レースの予測が完了しました")

## 6. 予測結果の表示

In [ ]:
def display_prediction(pred: Dict):
    """予測結果を表示"""
    print(f"\n{'='*60}")
    print(f"{pred['course']} {pred['race_name']}")
    print(f"レースID: {pred['race_id']}")
    print(f"{'='*60}")
    print(f"{'順位':>4} {'馬番':>4} {'馬名':<12} {'勝率':>8} {'オッズ':>8} {'期待値':>8}")
    print("-" * 60)
    
    for p in pred["predictions"][:5]:  # 上位5頭
        prob_pct = p["probability"] * 100
        odds_str = f"{p['odds']:.1f}" if p['odds'] else "-"
        ev = p["expected_value"]
        ev_mark = "★" if ev >= 1.0 else ""
        
        print(f"{p['pred_rank']:>4} {p['horse_number']:>4} {p['horse_name']:<12} {prob_pct:>7.1f}% {odds_str:>8} {ev:>7.2f}{ev_mark}")

# 各レースの予測を表示
for pred in all_predictions:
    display_prediction(pred)

## 7. 予測結果の保存（Supabase）

In [ ]:
import json

def save_predictions_to_supabase(predictions: List[Dict]) -> int:
    """予測結果をSupabaseに保存"""
    saved = 0
    
    for pred in predictions:
        try:
            data = {
                "race_id": pred["race_id"],
                "model_version": MODEL_VERSION,
                "results_json": pred["predictions"],
            }
            
            # 既存チェック
            existing = supabase.table("predictions").select("id").eq("race_id", pred["race_id"]).eq("model_version", MODEL_VERSION).execute()
            
            if existing.data:
                # 更新
                supabase.table("predictions").update(data).eq("id", existing.data[0]["id"]).execute()
            else:
                # 新規
                supabase.table("predictions").insert(data).execute()
            
            saved += 1
        except Exception as e:
            print(f"保存エラー: {pred['race_id']} - {e}")
    
    return saved

# 予測を保存
saved_count = save_predictions_to_supabase(all_predictions)
print(f"\n{saved_count}件の予測をSupabaseに保存しました")

---

## 使い方まとめ

### ハイブリッド構成の運用フロー

1. **ローカルFastAPI（重い処理）**
   - モデル学習を実行
   - 学習完了後、Supabase Storageに自動アップロード
   - `POST /api/v1/model/retrain-and-upload`

2. **このノートブック（軽い処理）**
   - Supabase Storageから最新モデルをダウンロード
   - 当日のレースデータを取得
   - 予測を実行
   - 結果をSupabaseに保存

### メリット
- Colabのタイムアウトを気にせずに重い学習が可能
- Colabは軽量な推論のみ → 無料枠で十分
- どこからでも最新モデルで予測可能